In [25]:
# Initialize
from factfinder.calculate import Calculate
from factfinder.median import Median
from dotenv import load_dotenv
import os
import pandas as pd
import logging

# pd.options.display.float_format = "{:,.18f}".format

try:
    env_path = "../.env"
    load_dotenv(dotenv_path=env_path)
except:
    print(".env file is missing ...")

logger = logging.getLogger()

calculate = Calculate(
    api_key=os.environ["API_KEY"], year=2010, source="acs", geography="2010_to_2020"
)

In [26]:
##INPUTS -- change here
pff_variable = "mdvl"
geotype = "CT20"
census_geoid_list = ["36005020100"]

In [27]:
# See all digits of ratio
ratio = calculate.geo.ratio
ratio.loc[ratio.geoid_ct2020.isin(census_geoid_list), :]

,geoid_ct2010,geoid_ct2020,ratio
136,36005020100,36005020100,1.0


In [28]:
# Get ranges and design factor from metadata
ranges = calculate.meta.median_ranges(pff_variable)
design_factor = calculate.meta.median_design_factor(pff_variable)
top_coding = calculate.meta.median_top_coding(pff_variable)
bottom_coding = calculate.meta.median_bottom_coding(pff_variable)

In [29]:
# Calculate inputs in 2020 geogs
logger.disabled = True
df = calculate.calculate_e_m_multiprocessing(list(ranges.keys()), geotype)
logger.disabled = False

In [30]:
# 3. create a pivot table with census_geoid as the index, and
# pff_variable as column names. df_pivoted.e -> the estimation dataframe
df_pivoted = df.loc[
    df.census_geoid.isin(census_geoid_list), ["census_geoid", "pff_variable", "e"]
].pivot(index="census_geoid", columns="pff_variable", values=["e"])
df_pivoted = df_pivoted.round(16)
df_pivoted

e                                                   \
pff_variable ov100t124 ov125t149 ov150t174 ov175t199 ov1milpl ov200t249   
census_geoid                                                              
36005020100        0.0       0.0       0.0       0.0      0.0       0.0   

                                                      ...                    \
pff_variable ov250t299 ov300t399 ov400t499 ov500t749  ... ovl25t29 ovl30t34   
census_geoid                                          ...                     
36005020100        0.0       0.0       0.0       0.0  ...      0.0      9.0   

                                                                             \
pff_variable ovl35t39 ovl40t49 ovl50t59 ovl60t69 ovl70t79 ovl80t89 ovl90t99   
census_geoid                                                                  
36005020100       0.0      0.0      0.0      0.0      0.0      0.0      0.0   

                     
pff_variable ovlu10  
census_geoid         
36005020100    11.0  

[1 rows x 24 columns]

In [31]:
def get_median_and_median_moe(ranges, row, DF, top_coding, bottom_coding):
    md = Median(ranges, row, DF, top_coding, bottom_coding)
    e = md.median
    m = md.median_moe
    return pd.Series({"e": e, "m": m})

In [32]:
results = df_pivoted.e.apply(
    lambda x: get_median_and_median_moe(
        ranges, x, design_factor, top_coding, bottom_coding
    ),
    axis=1,
)
results["census_geoid"] = df_pivoted.index
results = results.reset_index(drop=True)
results["pff_variable"] = pff_variable
results["geotype"] = geotype

DEBUG:root:N/2 is in range [0, 9999]
DEBUG:root:
C_-1: Cumulative frequency up to bin below N/2: 0.0
L_0: Lower boundary of median group: 0
F_0: Frequency within median group: 11.0
W_0: Width of median group: 9999
DEBUG:root:
MEDIAN: 9090.0

DEBUG:root:N/2 is in range [0, 9999]
DEBUG:root:
C_-1: Cumulative frequency up to bin below N/2: 0.0
L_0: Lower boundary of median group: 0
F_0: Frequency within median group: 11.0
W_0: Width of median group: 9999
DEBUG:root:
MEDIAN: 9090.0

DEBUG:root:N/2 is in range [0, 9999]
DEBUG:root:
C_-1: Cumulative frequency up to bin below N/2: 0.0
L_0: Lower boundary of median group: 0
F_0: Frequency within median group: 11.0
W_0: Width of median group: 9999
DEBUG:root:
MEDIAN: 9090.0

DEBUG:root:
MEDIAN STATS:
-----
Median = 9090.0
Median_MOE = nan
B = 20.0
se_50 = 57.05260730238365
p_lower = -7.052607302383649
p_upper = 107.05260730238365
lower_bin = 0
upper_bin = nan
first_non_zero_bin = 0

DISTRIBUTION:
-----
- [0, 9999]: 55.00000000000001
- [10000, 1